# Topic classification using cosine similarity on the word embeddings

## Preprocessing
Same as in the data_exploration notebook

In [3]:
from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals

from enum import Enum

import nltk

nltk.download('stopwords')
from nltk.corpus import stopwords
from spacy.lang.de.stop_words import STOP_WORDS

import re
import spacy

from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.luhn import LuhnSummarizer as Summarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words

[nltk_data] Downloading package stopwords to /home/delta/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
class Party(Enum):
    AFD = 0
    CDU = 1
    FDP = 2
    GRUENE = 3
    LINKE = 4
    SPD = 5

In [5]:
nlp = spacy.load('de_core_news_md')

# stopwords
nltk_stopwords = stopwords.words('german')

# build stopwords list
all_stopwords = list(set(STOP_WORDS) | set(nltk_stopwords))
with open('data_exploration/custom_stopwords.txt', 'r', encoding='utf-8') as f:
    all_stopwords += [line.strip() for line in f.readlines()]

# Load files
party_text = {}
for party in Party:
    all_stopwords.extend(['{}'.format(party.name.lower())])
    with open('resources/' + party.name + '.txt', encoding='utf-8', errors='ignore') as txt:
        file = " ".join(l for l in txt)
        # remove gender *
        file = re.sub(r'\*innen(\w*)\s', r'\1 ', file)
    party_text[party] = file

In [6]:
import gensim


def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    doc = nlp(" ".join(texts))
    texts_out = [token.lemma_ for token in doc if token.pos_ in allowed_postags and token.lemma_ not in all_stopwords]

    return texts_out


def prepare_data(party: Party):
    """
    Prepare data for topic classification.
    Split into sections and lemmatize.

    :param party: The party to get the text from
    :return: a dict containing the text and the lemmatized text seperated in sections
    """
    # get sections
    sections = re.split(r'\n\s\n', party_text[party])
    cleaned_sections = {}
    for section in sections:
        wordbag = gensim.utils.simple_preprocess(section)
        #lemmatize
        # wordbag = lemmatization(wordbag)
        cleaned_sections[section] = wordbag

    # return all cleaned sections except empty ones
    return dict(filter(lambda x: len(x[1]) > 0, cleaned_sections.items()))


## Create target clusters

In [7]:
import numpy as np
from sklearn import metrics
from transformers import BertTokenizer, TFBertModel

In [8]:
# based on lda from the data_exploration notebook
cluster_dict = {
    "UMWELT":
        ["umwelt", "klima", "klimaschutz", "ökologie", "co2", "co", "landwirtschaft", "klimakrise", "treibhaus",
         "emissionen", "ausbau", "innovative", "natur", "wasser", "landwirtschaft", "nachhaltig", "strom",
         "solar", "windkraft", "wasserkraft", "kohlekraft", "atomkraft", "kernenergie", "energiewende",
         "erneuerbar", "öl", "gas", "energie", "nachhaltig", "wasserstoff", "luftqualität", "umwelt", "wälder",
         "erneuerbare energie"],

    "WIRTSCHAFT":
        ["unternehmen", "selbstständige", "firma", "markt", "kapital", "finanzierung", "ezb", "banken",
         "staatsanleihen", "währungsfonds", "staatsanleihen", "kredit", "euro", "industrie", "schulden", "steuern",
         "konzern", "kapital", "finanzen"],

    "BILDUNG":
        ["student", "schüler", "schule", "gesamtschule", "lehrer", "universität", "lehre", "elternunabhängig",
         "bildung", "hochschulen", "wissenschaft", "bildung", "erasmus", "forschung", "lehre", "ausbildung",
         "weiterbildung", "aufstiegsmöglichkeit", "bildungsstandard", "innovationen", "bafög", "studium"],

    "GESELLSCHAFT":
        ["gesellschaft", "kultur", "freiheit", "privat", "kulturelle", "antisemitismus",
         "vorbild", "frauen", "familie", "identität", "gender", "sprache", "leben", "religion", "christentum", "islam",
         "diskriminierung", "menschenrechte", "kunst", "adoption"],

    "INNEN":
        ["schutz", "polizei", "schützen", "überwachung", "datenschutz", "sicherheit", "bundeswehr", "asyl",
         "integration", "migrant", "flüchtling", "immigrant", "toleranz", "zuwanderung", "asylbewerber",
         "krimminalität", "zuwanderung", "kontrolle", "bundespolizei", "gefahr", "terroristen", "gewalt"],

    "ARBEIT_UND_SOZIALES":
        ["rente", "harz4", "arbeitslosengeld", "pflege", "wohngeld", "familie", "arbeitslos", "sozial", "bauen",
         "wohnungen", "sozialbau", "kinder", "pflegen", "arbeitssuchende", "grundsicherung", "eltern", "jugendlich",
         "gesundheit", "arzt", "armut", "einkommen", "löhne", "tarifvertrag", "bürgergeld",
         "sozialstaat", "teilhabe", "bezahlbar"]
}

In [9]:
tokenizer = BertTokenizer.from_pretrained("dbmdz/bert-base-german-uncased", do_lower_case=True)
model = TFBertModel.from_pretrained("dbmdz/bert-base-german-uncased")

2022-03-04 13:04:19.959880: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-04 13:04:19.959944: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-04 13:04:19.959996: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (debian): /proc/driver/nvidia/version does not exist
2022-03-04 13:04:19.961845: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-04 13:04:20.098287: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 95545344 exceeds 10%

In [10]:
def convert_to_embeddings(cluster):
    idx = tokenizer.encode(cluster)
    idx = np.array(idx)[None, :]
    embedding = model(idx)
    tensor = np.array(embedding[0][0][1:-1])
    return tensor

## Classify text

In [11]:
 results = {}
for party in Party:
    # create list of vectors
    section_dict = prepare_data(party)
    mean_vectors = [convert_to_embeddings(txt).mean(0) for txt in section_dict.values()]

    feature_matrix = np.array(mean_vectors)

    # vector space for dict (maybe mean(0) ?)
    dic_vectors = {key: convert_to_embeddings(value) for key, value in cluster_dict.items()}

    similarities = np.array(
        [metrics.pairwise.cosine_similarity(feature_matrix, dic_y).T.tolist()[0] for dic_y in dic_vectors.values()]).T

    labels = list(dic_vectors.keys())
    for i in range(len(similarities)):

        # if there is a similarity to a cluster -> random cluster
        if sum(similarities[i]) == 0:
            similarities[i] = [0] * len(labels)
            similarities[i][np.random.choice(range(len(labels)))] = 1

        # rescale to 1
        similarities[i] = similarities[i] / sum(similarities[i])

    #classify based on similarities
    prediction = [(labels[np.argmax(pred)], max(pred)) for pred in similarities]

    # print number of topics for each topic
    print(party)
    for topic in labels:
        print(topic, ":", len(list(filter(lambda x: x[0] == topic, prediction))))
    print()

    results[party] = (section_dict, prediction)

Party.AFD
UMWELT : 12
WIRTSCHAFT : 103
BILDUNG : 2
GESELLSCHAFT : 82
INNEN : 210
ARBEIT_UND_SOZIALES : 7

Party.CDU
UMWELT : 98
WIRTSCHAFT : 400
BILDUNG : 14
GESELLSCHAFT : 156
INNEN : 452
ARBEIT_UND_SOZIALES : 7

Party.FDP
UMWELT : 41
WIRTSCHAFT : 91
BILDUNG : 13
GESELLSCHAFT : 31
INNEN : 190
ARBEIT_UND_SOZIALES : 7

Party.GRUENE
UMWELT : 45
WIRTSCHAFT : 108
BILDUNG : 1
GESELLSCHAFT : 15
INNEN : 130
ARBEIT_UND_SOZIALES : 8

Party.LINKE
UMWELT : 25
WIRTSCHAFT : 171
BILDUNG : 6
GESELLSCHAFT : 44
INNEN : 194
ARBEIT_UND_SOZIALES : 27

Party.SPD
UMWELT : 21
WIRTSCHAFT : 44
BILDUNG : 1
GESELLSCHAFT : 10
INNEN : 47
ARBEIT_UND_SOZIALES : 2



In [12]:
party_text_by_topic = {}
for party in Party:
    umwelt_text = ""
    wirtschaft_text = ""
    bildung_text = ""
    gesellschaft_text = ""
    innen_text = ""
    arbeit_text = ""
    texts = results.get(party)[0]
    classifications = results.get(party)[1]
    for text, clas in zip(texts, classifications):
        if clas[0] == 'UMWELT':
            umwelt_text += " " + text
        elif clas[0] == 'WIRTSCHAFT':
            wirtschaft_text += " " + text
        elif clas[0] == 'BILDUNG':
            bildung_text += " " + text
        elif clas[0] == 'GESELLSCHAFT':
            gesellschaft_text += " " + text
        elif clas[0] == 'INNEN':
            innen_text += " " + text
        elif clas[0] == 'ARBEIT_UND_SOZIALES':
            arbeit_text += " " + text
    party_text_by_topic[party] = [("UMWELT", umwelt_text), ("WIRTSCHAFT", wirtschaft_text), ("BILDUNG", bildung_text),
                                  ("GESELLSCHAFT", gesellschaft_text), ("INNEN", innen_text), ("ARBEIT", arbeit_text)]

print(party_text_by_topic)

{<Party.AFD: 0>: [('UMWELT', '  Der Linksextremismus überschreitet zunehmend die\n Schwelle zum Linksterrorismus. Wir fordern daher\n verstärkte Anstrengungen im Kampf gegen den\n Linksextremismus.  Eine existentielle Frage\n wie die Zuwanderung\n muss in demokratischer\n Selbstbestimmung\n auf nationaler Ebene\n entschieden werden.  Die AfD fordert:  Die AfD fordert, die\n Schönheit historischer\n Innenstädte zu bewahren und bei Bedarf\n durch Rekonstruktionen\n wiederherzustellen.  Das Ziel der Bundesregierung, die CO2-Emissionen\n faktisch auf null zu senken, führt zu einem radikalen\n Umbau von Industrie und Gesellschaft („Die Große\n Transformation“ / „The Great Reset“) und bedroht unsere\n Freiheit in einem immer beängstigenderen Ausmaß.\n Die AfD lehnt dieses Ziel und den damit verbundenen\n Gesellschaftsumbau ab.\n Begründet wird der radikale Umbau von Industrie\n und Gesellschaft mit der Behauptung, dass durch eine\n Dekarbonisierung, also den Verzicht auf die Nutzung\n von Ko

## Summarization variables

In [13]:
LANGUAGE = "german"
SENTENCES_COUNT = 10

In [14]:
def summarize(text):
    parser = PlaintextParser.from_string(text, Tokenizer(LANGUAGE))
    stemmer = Stemmer(LANGUAGE)
    summarizer = Summarizer(stemmer)
    summarizer.stop_words = get_stop_words(LANGUAGE)
    summarizer(parser.document, SENTENCES_COUNT)
    string = ""
    for sentence in summarizer(parser.document, SENTENCES_COUNT):
        string += " " + str(sentence)
        print(sentence)
    return string

## Summarize

In [15]:
summaries_by_topic = {}
for party in Party:
    party_summary = []
    for text in party_text_by_topic.get(party):
        summary = summarize(text[1])
        party_summary.append((text[0], summary))

    summaries_by_topic[party] = party_summary



Die AfD fordert:  Die AfD fordert, die Schönheit historischer Innenstädte zu bewahren und bei Bedarf durch Rekonstruktionen wiederherzustellen.
Das Ziel der Bundesregierung, die CO2-Emissionen faktisch auf null zu senken, führt zu einem radikalen Umbau von Industrie und Gesellschaft („Die Große Transformation“ / „The Great Reset“) und bedroht unsere Freiheit in einem immer beängstigenderen Ausmaß.
Die AfD lehnt dieses Ziel und den damit verbundenen Gesellschaftsumbau ab.
Das Ziel der Bundesregierung, die CO2-Emissionen faktisch auf null zu senken, lehnt die AfD ab.
Die AfD bezweifelt aber, dass diese nur negative Folgen hat.
Es zeigt sich, dass gebietsfremde Arten nicht nur negative Auswirkungen auf unsere hiesige Natur haben, sondern auch konkrete finanzielle und gesundheitliche Schäden verursachen.
Wir müssen größere Anstrengungen unternehmen, um die Einschleppung invasiver Tier- und Pflanzenarten zu verhindern und ihre Ausbreitung einzudämmen.
Der Wald soll von Windindustrieanlagen 

## Print summaries

In [16]:
for party in Party:
    information = summaries_by_topic.get(party)
    for item in information:
        print("PARTY: ", party.name, " TOPIC: ", item[0], " SUMMARY: ", item[1])
        print("\n")
    print("\n\n")

PARTY:  AFD  TOPIC:  UMWELT  SUMMARY:   Die AfD fordert:  Die AfD fordert, die Schönheit historischer Innenstädte zu bewahren und bei Bedarf durch Rekonstruktionen wiederherzustellen. Das Ziel der Bundesregierung, die CO2-Emissionen faktisch auf null zu senken, führt zu einem radikalen Umbau von Industrie und Gesellschaft („Die Große Transformation“ / „The Great Reset“) und bedroht unsere Freiheit in einem immer beängstigenderen Ausmaß. Die AfD lehnt dieses Ziel und den damit verbundenen Gesellschaftsumbau ab. Das Ziel der Bundesregierung, die CO2-Emissionen faktisch auf null zu senken, lehnt die AfD ab. Die AfD bezweifelt aber, dass diese nur negative Folgen hat. Es zeigt sich, dass gebietsfremde Arten nicht nur negative Auswirkungen auf unsere hiesige Natur haben, sondern auch konkrete finanzielle und gesundheitliche Schäden verursachen. Wir müssen größere Anstrengungen unternehmen, um die Einschleppung invasiver Tier- und Pflanzenarten zu verhindern und ihre Ausbreitung einzudämmen.